In [51]:
import tensorflow as tf
import pandas as pd

train_path = './dataset/train'
valid_path = './dataset/valid'

train_ds_3 = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
     label_mode = 'binary',
    #label_mode='categorical',
    validation_split=0.2,
    subset='training',
    seed=42,
    image_size=(224, 224),
    batch_size=32
)

test_ds_3 = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
     label_mode = 'binary',
    #label_mode='categorical',
    validation_split=0.2,
    subset='validation',
    seed=42,
    image_size=(224, 224),
    batch_size=32
)

Found 840 files belonging to 2 classes.
Using 672 files for training.
Found 840 files belonging to 2 classes.
Using 168 files for validation.


In [33]:
val_ds.class_names

['African', 'Asian']

In [34]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

In [35]:
base_model.trainable = False

In [36]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = tf.keras.layers.Rescaling(1./127.5, offset=-1)(inputs) # rescaling
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)  # 3차원(7, 7, 1280) --> 1차원(1280)으로 줄이기 : GlobalAveragePooling2D
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

In [37]:
model = tf.keras.Model(inputs=inputs, outputs=output)
model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_2 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [38]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),  # Optimization
              loss='binary_crossentropy',
              metrics=['accuracy'])             # Metrics / Accuracy

In [39]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

## EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

In [40]:
checkpoint_path = "my_checkpoint.keras"
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_best_only=True,
                             monitor='val_loss',
                             verbose=1)

In [68]:
history = model.fit(
    train_ds_3,
    validation_data = test_ds_3,
    epochs=100,
    callbacks=[es, checkpoint]
)


Epoch 1/100
19/21 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.7607 - loss: 0.5180
Epoch 1: val_loss improved from 0.54586 to 0.54503, saving model to my_checkpoint.keras
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.7605 - loss: 0.5182 - val_accuracy: 0.7500 - val_loss: 0.5450
Epoch 2/100
19/21 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.7849 - loss: 0.4910
Epoch 2: val_loss improved from 0.54503 to 0.54197, saving model to my_checkpoint.keras
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.7815 - loss: 0.4946 - val_accuracy: 0.7560 - val_loss: 0.5420
Epoch 3/100
19/21 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.7751 - loss: 0.4987
Epoch 3: val_loss improved from 0.54197 to 0.54186, saving model to my_checkpoint.keras
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.7728 - loss: 0.5009 - val_accuracy: 0.7560 - val_loss: 0.5419
Epoch 4/100
19/21 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.7508 - loss: 0.5228
Epoch 4: val_loss improved from 0.54186 to 0

In [69]:
classes = train_ds_3.class_names
classes

['African', 'Asian']

In [70]:
import os
file_list = os.listdir(valid_path)

In [50]:
#!mkdir ./dataset/valid
!cp -p ./dataset/test/African/* ./dataset/valid/
!cp -p ./dataset/test/Asian/* ./dataset/valid/

In [71]:
from keras.preprocessing import image
import numpy as np

results = []

for item in file_list:
  img_path = os.path.join(valid_path, item)
  img = image.load_img(img_path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  # x = preprocess_input(x)
  # x=x/255.0
  # print(x[0])

  preds = model.predict(x)
  print(item, preds)
  # preds=np.argmax(preds)

  if preds[0] > 0.5:
    results.append([item, classes[1]])
  else:
    results.append([item, classes[0]])



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
af_te69.jpg [[0.1568017]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
af_te12.jpg [[0.16435285]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
af_te48.jpg [[0.35820827]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
as_te81.jpg [[0.8805153]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
as_te2.jpg [[0.62526536]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
af_te13.jpg [[0.15992343]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
as_te57.jpg [[0.26854944]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
af_te83.jpg [[0.2089746]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
as_te71.jpg [[0.7635095]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
as_te34.jpg [[0.75093096]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
af_te66.jpg [[0.7409021]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
as_te49.jpg [[0.89336795]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
af_te85.jpg [[0.49431834]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
af_te27.jpg [[0.1522801]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
af_te1.jpg [[0.19694662]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/

In [75]:
# print(results)
df = pd.DataFrame(data = results, columns = ["filename", "class"])
  # preds=np.argmax(preds)
  # print('Predicted:', decode_predictions(preds))
  # print("filename : {}, class : {}".format(item, classes[preds]))
  # print(label_dict[str(preds)])

In [76]:
df

,filename,class
0,af_te69.jpg,African
1,af_te12.jpg,African
2,af_te48.jpg,African
3,as_te81.jpg,Asian
4,as_te2.jpg,Asian
...,...,...
183,af_te26.jpg,African
184,af_te37.jpg,African
185,af_te24.jpg,African
186,af_te56.jpg,African


In [77]:
mask = (df['filename'].str[:2] == 'af') & (df['class'] != 'African')

result = df[mask]
print(result.size)

30


In [ ]:
df.to_csv('result.csv', index=False)